In [5]:
import tensorflow as tf
def fire(x, squeeze, expand):
    y = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
    y = tf.keras.layers.BatchNormalization()(y)

    y1 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=1, activation='relu', padding='same')(y)
    y1 = tf.keras.layers.BatchNormalization()(y1)

    y3 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=3, activation='relu', padding='same')(y)
    y3 = tf.keras.layers.BatchNormalization()(y3)

    return tf.keras.layers.concatenate([y1, y3])


In [6]:
def fire_moudle(squeeze, expand):
    return lambda x: fire(x, squeeze, expand)

In [9]:
IMG_WIDHT = 224
IMG_HEIGHT = 224
IMG_CHANNELS = 3  

x = tf.keras.layers.Input(shape=(IMG_WIDHT, IMG_HEIGHT, IMG_CHANNELS))

y = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same')(x)
y = tf.keras.layers.BatchNormalization()(y)
y = fire_moudle(16, 32)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)

y = fire_moudle(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)

y = fire_moudle(64, 128)(y)
y = fire_moudle(80, 160)(y)
y = fire_moudle(96, 192)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)

y = fire_moudle(112, 224)(y)
y = fire_moudle(128, 256)(y)
y = fire_moudle(160, 320)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)

y = fire_moudle(192, 384)(y)
y = fire_moudle(224, 448)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)

y = fire_moudle(256, 512)(y)
y = tf.keras.layers.GlobalAveragePooling2D()(y)
y = tf.keras.layers.Dense(1000, activation='softmax')(y)

model = tf.keras.Model(inputs=x, outputs=y)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_66 (Conv2D)             (None, 224, 224, 32  896         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_64 (BatchN  (None, 224, 224, 32  128        ['conv2d_66[0][0]']              
 ormalization)                  )                                                             

In [11]:
model.save('squeeznet.h5')